# This code is to gather the information of the energy consumption of the whole training process of different models.

## import the required libraries

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import torchvision
import torchvision.transforms as transforms
from d2l import torch as d2l
import numpy as np
import pandas as pd
from ptflops import get_model_complexity_info
from pathlib import Path
import os
import time
import pynvml
import threading
import queue

## find the path

In [2]:
'''find the Model path'''
# find the current path
from pathlib import Path

# find the current path
current_path = Path.cwd()
print('The current path is:', current_path)

# find the data path
data_path = Path(current_path / 'ModelsData')
print('The data path is:', data_path)

The current path is: /root/autodl-tmp/GreenAI/3080
The data path is: /root/autodl-tmp/GreenAI/3080/ModelsData


## Models

### generate the data paths

In [3]:
models_name = ['alexnet', 
               'vgg11', 'vgg13', 'vgg16', 
               'resnet18', 'resnet34', 'resnet50',
               'googlenet_origin', 'googlenet_mod1', 'googlenet_mod2', 'googlenet_mod3',
               'googlenet_mod4', 'googlenet_mod5', 'googlenet_mod6', 'googlenet_mod7', 
               'googlenet_mod8', 'googlenet_mod9',
               'mobilenetv1_path', 'mobilenetv2_path']

In [4]:
DataList = [Path(f"{data_path}/{i}") for i in models_name]
print(DataList)

[PosixPath('/root/autodl-tmp/GreenAI/3080/ModelsData/alexnet'), PosixPath('/root/autodl-tmp/GreenAI/3080/ModelsData/vgg11'), PosixPath('/root/autodl-tmp/GreenAI/3080/ModelsData/vgg13'), PosixPath('/root/autodl-tmp/GreenAI/3080/ModelsData/vgg16'), PosixPath('/root/autodl-tmp/GreenAI/3080/ModelsData/resnet18'), PosixPath('/root/autodl-tmp/GreenAI/3080/ModelsData/resnet34'), PosixPath('/root/autodl-tmp/GreenAI/3080/ModelsData/resnet50'), PosixPath('/root/autodl-tmp/GreenAI/3080/ModelsData/googlenet_origin'), PosixPath('/root/autodl-tmp/GreenAI/3080/ModelsData/googlenet_mod1'), PosixPath('/root/autodl-tmp/GreenAI/3080/ModelsData/googlenet_mod2'), PosixPath('/root/autodl-tmp/GreenAI/3080/ModelsData/googlenet_mod3'), PosixPath('/root/autodl-tmp/GreenAI/3080/ModelsData/googlenet_mod4'), PosixPath('/root/autodl-tmp/GreenAI/3080/ModelsData/googlenet_mod5'), PosixPath('/root/autodl-tmp/GreenAI/3080/ModelsData/googlenet_mod6'), PosixPath('/root/autodl-tmp/GreenAI/3080/ModelsData/googlenet_mod7'),

### create the models

#### AlexNet Model

In [5]:
def alexnet(img_channel, num_labels):
    net = nn.Sequential(
        # 这里使用一个11*11的更大窗口来捕捉对象。
        # 同时，步幅为4，以减少输出的高度和宽度。
        # 另外，输出通道的数目远大于LeNet
        nn.Conv2d(img_channel, 64, kernel_size=11, stride=4, padding=2), nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2),
        # 减小卷积窗口，使用填充为2来使得输入与输出的高和宽一致，且增大输出通道数
        nn.Conv2d(64, 192, kernel_size=5, padding=2), nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2),
        # 使用三个连续的卷积层和较小的卷积窗口。
        # 除了最后的卷积层，输出通道的数量进一步增加。
        # 在前两个卷积层之后，汇聚层不用于减少输入的高度和宽度
        nn.Conv2d(192, 384, kernel_size=3, padding=1), nn.ReLU(),
        nn.Conv2d(384, 256, kernel_size=3, padding=1), nn.ReLU(),
        nn.Conv2d(256, 256, kernel_size=3, padding=1), nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.AdaptiveAvgPool2d((6, 6)),   # 使用全局平均池化对每个通道中所有元素求平均并直接将结果传递到全连接层
        nn.Flatten(),
        # 这里，全连接层的输出数量是LeNet中的好几倍。使用dropout层来减轻过拟合
        nn.Linear(256 * 6 * 6, 4096), nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(4096, 4096), nn.ReLU(),
        nn.Dropout(p=0.5),
        # 最后是输出层。由于这里使用Fashion-MNIST，所以用类别数为10，而非论文中的1000
        nn.Linear(4096, num_labels))
    return net

#### VGG Models

##### VGG11 Model

In [6]:
def vgg11_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)

def vgg11(input_channels, output_channels):
    conv_arch = [(1, 64), (1, 128), (2, 256), (2, 512), (2, 512)]
    conv_arch = [(1, 64), (1, 128), (2, 256), (2, 512), (2, 512)]
    in_channels = input_channels  # For RGB images
    # Create convolutional layers
    conv_layers = []
    for num_convs, out_channels in conv_arch:
        conv_layers.append(vgg11_block(num_convs, in_channels, out_channels))
        in_channels = out_channels

    return nn.Sequential(
        *conv_layers, nn.Flatten(),
        nn.Linear(out_channels * 7 * 7, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, output_channels)  # Output layer for 1000 classes
    )

##### VGG13 Model

In [7]:
def vgg13_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)

def vgg13(input_channels, output_channels):
    # VGG-13 architecture
    conv_arch = [(2, 64), (2, 128), (2, 256), (2, 512), (2, 512)]
    in_channels = input_channels  # For RGB images
    # Create convolutional layers
    conv_layers = []
    for num_convs, out_channels in conv_arch:
        conv_layers.append(vgg13_block(num_convs, in_channels, out_channels))
        in_channels = out_channels

    return nn.Sequential(
        *conv_layers, nn.Flatten(),
        nn.Linear(out_channels * 7 * 7, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, output_channels)  # Output layer for 1000 classes
    )

##### VGG16 Model

In [8]:
def vgg16_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)

def vgg16(input_channels, output_channels):
    conv_arch = [(2, 64), (2, 128), (3, 256), (3, 512), (3, 512)]
    in_channels = input_channels  # For RGB images
    # Create convolutional layers
    conv_layers = []
    for num_convs, out_channels in conv_arch:
        conv_layers.append(vgg16_block(num_convs, in_channels, out_channels))
        in_channels = out_channels

    return nn.Sequential(
        *conv_layers, nn.Flatten(),
        nn.Linear(out_channels * 7 * 7, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, output_channels)  # Output layer for 1000 classes
    )

#### ResNet Models

##### ResNet18 Model

In [9]:
class Residual18(nn.Module):  #@save
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,
                               kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels,
                               kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()

    def forward(self, X):
        Y = self.relu1(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return self.relu2(Y)
    
    
def resnet18(img_channel, num_labels):
    # blk = Residual(3,6, use_1x1conv=True, strides=2)

    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                    nn.BatchNorm2d(64), nn.ReLU(),
                    nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

    def resnet_block(input_channels, num_channels, num_residuals,
                    first_block=False):
        blk = []
        for i in range(num_residuals):
            if i == 0 and not first_block:
                blk.append(Residual18(input_channels, num_channels,
                                    use_1x1conv=True, strides=2))
            else:
                blk.append(Residual18(num_channels, num_channels))
        return blk

    b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
    b3 = nn.Sequential(*resnet_block(64, 128, 2))
    b4 = nn.Sequential(*resnet_block(128, 256, 2))
    b5 = nn.Sequential(*resnet_block(256, 512, 2))

    net = nn.Sequential(b1, b2, b3, b4, b5,
                        nn.AdaptiveAvgPool2d((1,1)),
                        nn.Flatten(), nn.Linear(512, num_labels))
    return net

##### ResNet34 Model

In [10]:
class Residual34(nn.Module):  #@save
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,
                               kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels,
                               kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)
    
    
def resnet34(img_channel, num_labels):
    # blk = Residual(3,6, use_1x1conv=True, strides=2)

    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                    nn.BatchNorm2d(64), nn.ReLU(),
                    nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

    def resnet_block(input_channels, num_channels, num_residuals,
                    first_block=False):
        blk = []
        for i in range(num_residuals):
            if i == 0 and not first_block:
                blk.append(Residual34(input_channels, num_channels,
                                    use_1x1conv=True, strides=2))
            else:
                blk.append(Residual34(num_channels, num_channels))
        return blk

    b2 = nn.Sequential(*resnet_block(64, 64, 3, first_block=True))
    b3 = nn.Sequential(*resnet_block(64, 128, 4))
    b4 = nn.Sequential(*resnet_block(128, 256, 6))
    b5 = nn.Sequential(*resnet_block(256, 512, 3))

    net = nn.Sequential(b1, b2, b3, b4, b5,
                        nn.AdaptiveAvgPool2d((1,1)),
                        nn.Flatten(), nn.Linear(512, num_labels))
    return net

##### ResNet50 Model

In [11]:
class Residual50(nn.Module):
    def __init__(self, input_channels, num_channels, use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels , kernel_size=1, stride=strides, bias=False)
        self.bn1 = nn.BatchNorm2d(num_channels )
        self.conv2 = nn.Conv2d(num_channels , num_channels , kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(num_channels )
        self.conv3 = nn.Conv2d(num_channels , num_channels, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(num_channels)
        
        if use_1x1conv or strides != 1:
            self.downsample = nn.Sequential(
                nn.Conv2d(input_channels, num_channels, kernel_size=1, stride=strides, bias=False),
                nn.BatchNorm2d(num_channels)
            )
        else:
            self.downsample = None

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = F.relu(self.bn2(self.conv2(Y)))
        Y = self.bn3(self.conv3(Y))
        if self.downsample:
            X = self.downsample(X)
        Y += X
        return F.relu(Y)

def resnet50(img_channel, num_labels):
    b1 = nn.Sequential(
        nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
        nn.BatchNorm2d(64), nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    )

    def resnet_block(input_channels, num_channels, num_residuals, first_block=False):
        blk = []
        for i in range(num_residuals):
            if i == 0 and not first_block:
                blk.append(Residual50(input_channels, num_channels, use_1x1conv=True, strides=2))
            else:
                blk.append(Residual50(num_channels, num_channels))
        return blk

    b2 = nn.Sequential(*resnet_block(64, 64, 3, first_block=True))
    b3 = nn.Sequential(*resnet_block(64, 128, 4))
    b4 = nn.Sequential(*resnet_block(128, 256, 6))
    b5 = nn.Sequential(*resnet_block(256, 512, 3))

    net = nn.Sequential(
        b1, b2, b3, b4, b5,
        nn.AdaptiveAvgPool2d((1, 1)),
        nn.Flatten(), nn.Linear(512, num_labels)
    )
    return net

#### GoogleNet Models

##### GoogleNet Model(orinigal)

In [12]:
class Inception(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 线路2，1x1卷积层后接3x3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1x1卷积层后接5x5卷积层
        self.p3_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3x3最大汇聚层后接1x1卷积层
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        # 在通道维度上连结输出
        return torch.cat((p1, p2, p3, p4), dim=1)
    
def Googlenet(img_channel, num_labels):
    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b3 = nn.Sequential(Inception(192, 64, (96, 128), (16, 32), 32),
                   Inception(256, 128, (128, 192), (32, 96), 64),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b4 = nn.Sequential(Inception(480, 192, (96, 208), (16, 48), 64),
                   Inception(512, 160, (112, 224), (24, 64), 64),
                   Inception(512, 128, (128, 256), (24, 64), 64),
                   Inception(512, 112, (144, 288), (32, 64), 64),
                   Inception(528, 256, (160, 320), (32, 128), 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b5 = nn.Sequential(Inception(832, 256, (160, 320), (32, 128), 128),
                   Inception(832, 384, (192, 384), (48, 128), 128),
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

    net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(1024, num_labels))
    return net

##### GoogleNet modified version1 Model

In [13]:
class Inception_mod1(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c1, c2, c3, c4, **kwargs):
        super(Inception_mod1, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        # self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 线路2，1x1卷积层后接3x3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1x1卷积层后接5x5卷积层
        self.p3_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3x3最大汇聚层后接1x1卷积层
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)

    def forward(self, x):
        # p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        # 在通道维度上连结输出
        return torch.cat((p2, p3, p4), dim=1)
    
def Googlenet_mod1(img_channel, num_labels):
    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b3 = nn.Sequential(Inception_mod1(192, 64, (96, 128), (16, 32), 32),
                   Inception_mod1(256-64, 128, (128, 192), (32, 96), 64),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b4 = nn.Sequential(Inception_mod1(480-128, 192, (96, 208), (16, 48), 64),
                   Inception_mod1(512-192, 160, (112, 224), (24, 64), 64),
                   Inception_mod1(512-160, 128, (128, 256), (24, 64), 64),
                   Inception_mod1(512-128, 112, (144, 288), (32, 64), 64),
                   Inception_mod1(528-112, 256, (160, 320), (32, 128), 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b5 = nn.Sequential(Inception_mod1(832-256, 256, (160, 320), (32, 128), 128),
                   Inception_mod1(832-256, 384, (192, 384), (48, 128), 128),
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

    net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(1024-384, num_labels))
    return net

##### GoogleNet modified version2 Model

In [14]:
class Inception_mod2(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c1, c2, c3, c4, **kwargs):
        super(Inception_mod2, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 线路2，1x1卷积层后接3x3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1x1卷积层后接5x5卷积层
        self.p3_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3x3最大汇聚层后接1x1卷积层
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        # p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        # 在通道维度上连结输出
        return torch.cat((p1, p3, p4), dim=1)
    
def Googlenet_mod2(img_channel, num_labels):
    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b3 = nn.Sequential(Inception_mod2(192, 64, (96, 128), (16, 32), 32),
                   Inception_mod2(256-128, 128, (128, 192), (32, 96), 64),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b4 = nn.Sequential(Inception_mod2(480-192, 192, (96, 208), (16, 48), 64),
                   Inception_mod2(512-208, 160, (112, 224), (24, 64), 64),
                   Inception_mod2(512-224, 128, (128, 256), (24, 64), 64),
                   Inception_mod2(512-256, 112, (144, 288), (32, 64), 64),
                   Inception_mod2(528-288, 256, (160, 320), (32, 128), 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b5 = nn.Sequential(Inception_mod2(832-320, 256, (160, 320), (32, 128), 128),
                   Inception_mod2(832-320, 384, (192, 384), (48, 128), 128),
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

    net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(1024-384, num_labels))
    return net

##### GoogleNet modified version3 Model

In [15]:
class Inception_mod3(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c1, c2, c3, c4, **kwargs):
        super(Inception_mod3, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 线路2，1x1卷积层后接3x3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1x1卷积层后接5x5卷积层
        # self.p3_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        # self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3x3最大汇聚层后接1x1卷积层
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        # p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        # 在通道维度上连结输出
        return torch.cat((p1, p2, p4), dim=1)
    
def Googlenet_mod3(img_channel, num_labels):
    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b3 = nn.Sequential(Inception_mod3(192, 64, (96, 128), (16, 32), 32),
                   Inception_mod3(256-32, 128, (128, 192), (32, 96), 64),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b4 = nn.Sequential(Inception_mod3(480-96, 192, (96, 208), (16, 48), 64),
                   Inception_mod3(512-48, 160, (112, 224), (24, 64), 64),
                   Inception_mod3(512-64, 128, (128, 256), (24, 64), 64),
                   Inception_mod3(512-64, 112, (144, 288), (32, 64), 64),
                   Inception_mod3(528-64, 256, (160, 320), (32, 128), 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b5 = nn.Sequential(Inception_mod3(832-128, 256, (160, 320), (32, 128), 128),
                   Inception_mod3(832-128, 384, (192, 384), (48, 128), 128),
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

    net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(1024-128, num_labels))
    return net

##### GoogleNet modified version4 Model

In [16]:
class Inception_mod4(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c1, c2, c3, c4, **kwargs):
        super(Inception_mod4, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 线路2，1x1卷积层后接3x3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1x1卷积层后接5x5卷积层
        self.p3_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3x3最大汇聚层后接1x1卷积层
        # self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        # self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        # p4 = F.relu(self.p4_2(self.p4_1(x)))
        # 在通道维度上连结输出
        return torch.cat((p1, p2, p3), dim=1)
    
def Googlenet_mod4(img_channel, num_labels):
    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b3 = nn.Sequential(Inception_mod4(192, 64, (96, 128), (16, 32), 32),
                   Inception_mod4(256-32, 128, (128, 192), (32, 96), 64),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b4 = nn.Sequential(Inception_mod4(480-64, 192, (96, 208), (16, 48), 64),
                   Inception_mod4(512-64, 160, (112, 224), (24, 64), 64),
                   Inception_mod4(512-64, 128, (128, 256), (24, 64), 64),
                   Inception_mod4(512-64, 112, (144, 288), (32, 64), 64),
                   Inception_mod4(528-64, 256, (160, 320), (32, 128), 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b5 = nn.Sequential(Inception_mod4(832-128, 256, (160, 320), (32, 128), 128),
                   Inception_mod4(832-128, 384, (192, 384), (48, 128), 128),
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

    net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(1024-128, num_labels))
    return net

##### GoogleNet modified version5 Model

In [17]:
class Inception_mod5(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c2, **kwargs):
        super(Inception_mod5, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        # self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 线路2，1x1卷积层后接3x3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1x1卷积层后接5x5卷积层
        # self.p3_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        # self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3x3最大汇聚层后接1x1卷积层
        # self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        # self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)

    def forward(self, x):
        # p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        # p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        # p4 = F.relu(self.p4_2(self.p4_1(x)))
        # 在通道维度上连结输出
        return torch.cat([p2], dim=1)
    
def Googlenet_mod5(img_channel, num_labels):
    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b3 = nn.Sequential(Inception_mod5(192, (96, 128)),
                   Inception_mod5(128, (128, 192)),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b4 = nn.Sequential(Inception_mod5(192, (96, 208)),
                   Inception_mod5(208, (112, 224)),
                   Inception_mod5(224, (128, 256)),
                   Inception_mod5(256, (144, 288)),
                   Inception_mod5(288, (160, 320)),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b5 = nn.Sequential(Inception_mod5(320, (160, 320)),
                   Inception_mod5(320, (192, 384)),
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

    net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(384, num_labels))
    return net

##### GoogleNet modified version6 Model

In [18]:
class Inception_mod6(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c2, **kwargs):
        super(Inception_mod6, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        # self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 线路2，1x1卷积层后接3x3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1x1卷积层后接5x5卷积层
        self.p3_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路4，3x3最大汇聚层后接1x1卷积层
        # self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        # self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)

    def forward(self, x):
        # p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        # p4 = F.relu(self.p4_2(self.p4_1(x)))
        # 在通道维度上连结输出
        return torch.cat((p2, p3), dim=1)
    
def Googlenet_mod6(img_channel, num_labels):
    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b3 = nn.Sequential(Inception_mod6(192, (96, 128)),
                   Inception_mod6(128*2, (128, 192)),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b4 = nn.Sequential(Inception_mod6(192*2, (96, 208)),
                   Inception_mod6(208*2, (112, 224)),
                   Inception_mod6(224*2, (128, 256)),
                   Inception_mod6(256*2, (144, 288)),
                   Inception_mod6(288*2, (160, 320)),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b5 = nn.Sequential(Inception_mod6(320*2, (160, 320)),
                   Inception_mod6(320*2, (192, 384)),
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

    net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(384*2, num_labels))
    return net

##### GoogleNet modified version7 Model

In [19]:
class Inception_mod7(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c2, **kwargs):
        super(Inception_mod7, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        # self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 线路2，1x1卷积层后接3x3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1x1卷积层后接5x5卷积层
        self.p3_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路4，3x3最大汇聚层后接1x1卷积层
        self.p4_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p4_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)

    def forward(self, x):
        # p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(F.relu(self.p4_1(x))))
        # 在通道维度上连结输出
        return torch.cat((p2, p3, p4), dim=1)
    
def Googlenet_mod7(img_channel, num_labels):
    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b3 = nn.Sequential(Inception_mod7(192, (96, 128)),
                   Inception_mod7(128*3, (128, 192)),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b4 = nn.Sequential(Inception_mod7(192*3, (96, 208)),
                   Inception_mod7(208*3, (112, 224)),
                   Inception_mod7(224*3, (128, 256)),
                   Inception_mod7(256*3, (144, 288)),
                   Inception_mod7(288*3, (160, 320)),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b5 = nn.Sequential(Inception_mod7(320*3, (160, 320)),
                   Inception_mod7(320*3, (192, 384)),
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

    net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(384*3, num_labels))
    return net

##### GoogleNet modified version8 Model

In [20]:
class Inception_mod8(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c2, **kwargs):
        super(Inception_mod8, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        self.p1_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p1_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路2，1x1卷积层后接3x3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1x1卷积层后接5x5卷积层
        self.p3_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路4，3x3最大汇聚层后接1x1卷积层
        self.p4_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p4_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)

    def forward(self, x):
        p1 = F.relu(self.p1_2(F.relu(self.p1_1(x))))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(F.relu(self.p4_1(x))))
        # 在通道维度上连结输出
        return torch.cat((p1, p2, p3, p4), dim=1)
    
def Googlenet_mod8(img_channel, num_labels):
    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b3 = nn.Sequential(Inception_mod8(192, (96, 128)),
                   Inception_mod8(128*4, (128, 192)),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b4 = nn.Sequential(Inception_mod8(192*4, (96, 208)),
                   Inception_mod8(208*4, (112, 224)),
                   Inception_mod8(224*4, (128, 256)),
                   Inception_mod8(256*4, (144, 288)),
                   Inception_mod8(288*4, (160, 320)),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b5 = nn.Sequential(Inception_mod8(320*4, (160, 320)),
                   Inception_mod8(320*4, (192, 384)),
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

    net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(384*4, num_labels))
    return net

##### GoogleNet modified version9 Model

In [21]:
# class Inception_mod8(nn.Module):
#     # c1--c4是每条路径的输出通道数
#     def __init__(self, in_channels, c2, **kwargs):
#         super(Inception_mod8, self).__init__(**kwargs)
#         # 线路1，单1x1卷积层
#         self.p1_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
#         self.p1_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
#         # 线路2，1x1卷积层后接3x3卷积层
#         self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
#         self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
#         # 线路3，1x1卷积层后接5x5卷积层
#         self.p3_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
#         self.p3_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
#         # 线路4，3x3最大汇聚层后接1x1卷积层
#         self.p4_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
#         self.p4_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)

#     def forward(self, x):
#         p1 = F.relu(self.p1_2(F.relu(self.p1_1(x))))
#         p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
#         p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
#         p4 = F.relu(self.p4_2(F.relu(self.p4_1(x))))
#         # 在通道维度上连结输出
#         return torch.cat((p1, p2, p3, p4), dim=1)
    
def Googlenet_mod9(img_channel, num_labels):
    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    # b3 = nn.Sequential(Inception_mod8(192, (96, 128)),
    #                Inception_mod8(128*4, (128, 192)),
    #                nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    # b4 = nn.Sequential(Inception_mod8(192*4, (96, 208)),
    #                Inception_mod8(208*4, (112, 224)),
    #                Inception_mod8(224*4, (128, 256)),
    #                Inception_mod8(256*4, (144, 288)),
    #                Inception_mod8(288*4, (160, 320)),
    #                nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    # b5 = nn.Sequential(Inception_mod8(320*4, (160, 320)),
    #                Inception_mod8(320*4, (192, 384)),
    #                nn.AdaptiveAvgPool2d((1,1)),
    #                nn.Flatten())
    
    b5 = nn.Sequential(
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

    net = nn.Sequential(b1, b2, b5, nn.Linear(192, num_labels))
    return net

#### MobileNet Models

##### MobileNetV1 Model

In [22]:
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(DepthwiseSeparableConv, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

class MobileNet(nn.Module):
    def __init__(self, input_channels, output_channels):
        super(MobileNet, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(input_channels, 32, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),

            DepthwiseSeparableConv(32, 64, 1),
            DepthwiseSeparableConv(64, 128, 2),
            DepthwiseSeparableConv(128, 128, 1),
            DepthwiseSeparableConv(128, 256, 2),
            DepthwiseSeparableConv(256, 256, 1),
            DepthwiseSeparableConv(256, 512, 2),

            # Typically, 5 Depthwise Separable Convolutions are repeated here, each with stride 1
            *[DepthwiseSeparableConv(512, 512, 1) for _ in range(5)],

            DepthwiseSeparableConv(512, 1024, 2),
            DepthwiseSeparableConv(1024, 1024, 1),

            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(1024, output_channels)
        )

    def forward(self, x):
        return self.model(x)

##### MobileNetV2 Model

In [23]:
class InvertedResidual(nn.Module):
    def __init__(self, in_channels, out_channels, stride, expand_ratio):
        super(InvertedResidual, self).__init__()
        self.stride = stride
        hidden_dim = in_channels * expand_ratio
        self.use_residual = self.stride == 1 and in_channels == out_channels

        self.layers = nn.Sequential(
            # expand
            nn.Conv2d(in_channels, hidden_dim, 1, 1, 0, bias=False),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU6(inplace=True),

            # depthwise
            nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU6(inplace=True),

            # project
            nn.Conv2d(hidden_dim, out_channels, 1, 1, 0, bias=False),
            nn.BatchNorm2d(out_channels)
        )

    def forward(self, x):
        if self.use_residual:
            return x + self.layers(x)
        else:
            return self.layers(x)

class MobileNetV2(nn.Module):
    def __init__(self, input_channels, output_channels):
        super(MobileNetV2, self).__init__()
        self.first_layer = nn.Sequential(
            nn.Conv2d(input_channels, 32, 3, 2, 1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU6(inplace=True)
        )

        self.inverted_residual_blocks = nn.Sequential(
            InvertedResidual(32, 16, 1, 1),
            InvertedResidual(16, 24, 2, 6),
            InvertedResidual(24, 24, 1, 6),
            InvertedResidual(24, 32, 2, 6),
            InvertedResidual(32, 32, 1, 6),
            InvertedResidual(32, 32, 1, 6),
            InvertedResidual(32, 64, 2, 6),
            InvertedResidual(64, 64, 1, 6),
            InvertedResidual(64, 64, 1, 6),
            InvertedResidual(64, 64, 1, 6),
            InvertedResidual(64, 96, 1, 6),
            InvertedResidual(96, 96, 1, 6),
            InvertedResidual(96, 96, 1, 6),
            InvertedResidual(96, 160, 2, 6),
            InvertedResidual(160, 160, 1, 6),
            InvertedResidual(160, 160, 1, 6),
            InvertedResidual(160, 320, 1, 6)
        )

        self.last_layers = nn.Sequential(
            nn.Conv2d(320, 1280, 1, 1, 0, bias=False),
            nn.BatchNorm2d(1280),
            nn.ReLU6(inplace=True),
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(1280, output_channels)
        )

    def forward(self, x):
        x = self.first_layer(x)
        x = self.inverted_residual_blocks(x)
        x = self.last_layers(x)
        return x

## Call the Models

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'mps')
print('The device is:', device)

# check if mps on macbook is availabel
# print(torch.backends.mps.is_available())  # 检查 MPS 是否可用
# print(torch.backends.mps.is_built())      # 检查 MPS 是否已编译

The device is: cuda


In [25]:
# create list to store all the parameters and the number of MACs, be careful of the different datasets
# to avoid the error of the number of input channels and any other mistake, try to use different dictionaries to store each dataset
# create different empty dictionary
macs_f = {}
paras_f = {}
macs_c100 = {}
paras_c100 = {}
macs_c10 = {}
paras_c10 = {}

### usea function to call the models

In [26]:
# create a function for all the models to run
# image channel for fashion mnist 
channel_f = 1
# image channel for cifar100 and cifar10
channel_c = 3

# number of labels for fashion mnist
num_labels_f = 10
# number of labels for cifar100 
num_labels_c100 = 100
# number of labels for cifar10
num_labels_c10 = 10

def get_model_info(model, img_channel, num_labels):
    model_ini = model.__name__
    print(f'The model name is {model_ini}')

    net = model(img_channel, num_labels)
    
    return net

### import all models

In [27]:
# alexnet
alexnet_f= get_model_info(alexnet, channel_f, num_labels_f)
alexnet_c100 = get_model_info(alexnet, channel_c, num_labels_c100)
alexnet_c10 = get_model_info(alexnet, channel_c, num_labels_c10)

# ResNet
# resnet18
resnet18_f = get_model_info(resnet18, channel_f, num_labels_f)
resnet18_c100 = get_model_info(resnet18, channel_c, num_labels_c100)
resnet18_c10 = get_model_info(resnet18, channel_c, num_labels_c10)
# resnet34
resnet34_f = get_model_info(resnet34, channel_f, num_labels_f)
resnet34_c100 = get_model_info(resnet34, channel_c, num_labels_c100)
resnet34_c10 = get_model_info(resnet34, channel_c, num_labels_c10)
# resnet50
resnet50_f = get_model_info(resnet50, channel_f, num_labels_f)
resnet50_c100 = get_model_info(resnet50, channel_c, num_labels_c100)
resnet50_c10 = get_model_info(resnet50, channel_c, num_labels_c10)

# VGG
# vgg11
vgg11_f = get_model_info(vgg11, channel_f, num_labels_f)
vgg11_c100 = get_model_info(vgg11, channel_c, num_labels_c100)
vgg11_c10 = get_model_info(vgg11, channel_c, num_labels_c10)
# vgg13
vgg13_f = get_model_info(vgg13, channel_f, num_labels_f)
vgg13_c100 = get_model_info(vgg13, channel_c, num_labels_c100)
vgg13_c10 = get_model_info(vgg13, channel_c, num_labels_c10)
# vgg16 
vgg16_f = get_model_info(vgg16, channel_f, num_labels_f)
vgg16_c100 = get_model_info(vgg16, channel_c, num_labels_c100)
vgg16_c10 = get_model_info(vgg16, channel_c, num_labels_c10)

# GoogLeNet
# googlenet_original
googlenet_org_f = get_model_info(Googlenet, channel_f, num_labels_f)
googlenet_org_c100 = get_model_info(Googlenet, channel_c, num_labels_c100)
googlenet_org_c10 = get_model_info(Googlenet, channel_c, num_labels_c10)

# googlenet_mod1
googlenet_mod1_f = get_model_info(Googlenet_mod1, channel_f, num_labels_f)
googlenet_mod1_c100 = get_model_info(Googlenet_mod1, channel_c, num_labels_c100)
googlenet_mod1_c10 = get_model_info(Googlenet_mod1, channel_c, num_labels_c10)

# googlenet_mod2
googlenet_mod2_f = get_model_info(Googlenet_mod2, channel_f, num_labels_f)
googlenet_mod2_c100 = get_model_info(Googlenet_mod2, channel_c, num_labels_c100)
googlenet_mod2_c10 = get_model_info(Googlenet_mod2, channel_c, num_labels_c10)

# googlenet_mod3
googlenet_mod3_f = get_model_info(Googlenet_mod3, channel_f, num_labels_f)
googlenet_mod3_c100 = get_model_info(Googlenet_mod3, channel_c, num_labels_c100)
googlenet_mod3_c10 = get_model_info(Googlenet_mod3, channel_c, num_labels_c10)

# googlenet_mod4
googlenet_mod4_f = get_model_info(Googlenet_mod4, channel_f, num_labels_f)
googlenet_mod4_c100 = get_model_info(Googlenet_mod4, channel_c, num_labels_c100)
googlenet_mod4_c10 = get_model_info(Googlenet_mod4, channel_c, num_labels_c10)

# googlenet_mod5
googlenet_mod5_f = get_model_info(Googlenet_mod5, channel_f, num_labels_f)
googlenet_mod5_c100 = get_model_info(Googlenet_mod5, channel_c, num_labels_c100)
googlenet_mod5_c10 = get_model_info(Googlenet_mod5, channel_c, num_labels_c10)

# googlenet_mod6
googlenet_mod6_f = get_model_info(Googlenet_mod6, channel_f, num_labels_f)
googlenet_mod6_c100 = get_model_info(Googlenet_mod6, channel_c, num_labels_c100)
googlenet_mod6_c10 = get_model_info(Googlenet_mod6, channel_c, num_labels_c10)

# googlenet_mod7
googlenet_mod7_f = get_model_info(Googlenet_mod7, channel_f, num_labels_f)
googlenet_mod7_c100 = get_model_info(Googlenet_mod7, channel_c, num_labels_c100)
googlenet_mod7_c10 = get_model_info(Googlenet_mod7, channel_c, num_labels_c10)

# googlenet_mod8
googlenet_mod8_f = get_model_info(Googlenet_mod8, channel_f, num_labels_f)
googlenet_mod8_c100 = get_model_info(Googlenet_mod8, channel_c, num_labels_c100)
googlenet_mod8_c10 = get_model_info(Googlenet_mod8, channel_c, num_labels_c10)

# googlenet_mod9
googlenet_mod9_f = get_model_info(Googlenet_mod9, channel_f, num_labels_f)
googlenet_mod9_c100 = get_model_info(Googlenet_mod9, channel_c, num_labels_c100)
googlenet_mod9_c10 = get_model_info(Googlenet_mod9, channel_c, num_labels_c10)

# MobileNet
# mobilenetv1
mobilenetv1_f = get_model_info(MobileNet, channel_f, num_labels_f)
mobilenetv1_c100 = get_model_info(MobileNet, channel_c, num_labels_c100)
mobilenetv1_c10 = get_model_info(MobileNet, channel_c, num_labels_c10)

# mobilenetv2
mobilenetv2_f = get_model_info(MobileNetV2, channel_f, num_labels_f)
mobilenetv2_c100 = get_model_info(MobileNetV2, channel_c, num_labels_c100)
mobilenetv2_c10 = get_model_info(MobileNetV2, channel_c, num_labels_c10)

The model name is alexnet


The model name is alexnet
The model name is alexnet
The model name is resnet18
The model name is resnet18
The model name is resnet18
The model name is resnet34
The model name is resnet34
The model name is resnet34
The model name is resnet50
The model name is resnet50
The model name is resnet50
The model name is vgg11
The model name is vgg11
The model name is vgg11
The model name is vgg13
The model name is vgg13
The model name is vgg13
The model name is vgg16
The model name is vgg16
The model name is vgg16
The model name is Googlenet
The model name is Googlenet
The model name is Googlenet
The model name is Googlenet_mod1
The model name is Googlenet_mod1
The model name is Googlenet_mod1
The model name is Googlenet_mod2
The model name is Googlenet_mod2
The model name is Googlenet_mod2
The model name is Googlenet_mod3
The model name is Googlenet_mod3
The model name is Googlenet_mod3
The model name is Googlenet_mod4
The model name is Googlenet_mod4
The model name is Googlenet_mod4
The model

## Import the Datasets for training

In [28]:
print(models_name)

['alexnet', 'vgg11', 'vgg13', 'vgg16', 'resnet18', 'resnet34', 'resnet50', 'googlenet_origin', 'googlenet_mod1', 'googlenet_mod2', 'googlenet_mod3', 'googlenet_mod4', 'googlenet_mod5', 'googlenet_mod6', 'googlenet_mod7', 'googlenet_mod8', 'googlenet_mod9', 'mobilenetv1_path', 'mobilenetv2_path']


In [29]:
# create model list according to models_name order
models_f_list = [alexnet_f, 
                vgg11_f, vgg13_f, vgg16_f,
                resnet18_f, resnet34_f, resnet50_f,
                googlenet_org_f, googlenet_mod1_f, googlenet_mod2_f, googlenet_mod3_f, 
                googlenet_mod4_f, googlenet_mod5_f, googlenet_mod6_f, googlenet_mod7_f,
                googlenet_mod8_f, googlenet_mod9_f,
                mobilenetv1_f, mobilenetv2_f]

models_c100_list = [alexnet_c100,
                vgg11_c100, vgg13_c100, vgg16_c100,
                resnet18_c100, resnet34_c100, resnet50_c100,
                googlenet_org_c100, googlenet_mod1_c100, googlenet_mod2_c100, googlenet_mod3_c100, 
                googlenet_mod4_c100, googlenet_mod5_c100, googlenet_mod6_c100, googlenet_mod7_c100,
                googlenet_mod8_c100, googlenet_mod9_c100,
                mobilenetv1_c100, mobilenetv2_c100]

models_c10_list = [alexnet_c10,
                vgg11_c10, vgg13_c10, vgg16_c10,
                resnet18_c10, resnet34_c10, resnet50_c10,
                googlenet_org_c10, googlenet_mod1_c10, googlenet_mod2_c10, googlenet_mod3_c10, 
                googlenet_mod4_c10, googlenet_mod5_c10, googlenet_mod6_c10, googlenet_mod7_c10,
                googlenet_mod8_c10, googlenet_mod9_c10,
                mobilenetv1_c10, mobilenetv2_c10]

### show the output size of each layers after the picture is passed through the model

In [30]:
X_f = torch.randn(size=(1, 1, 224, 224), dtype=torch.float32) # fashion mnist

# for layer in alexnet_f:
#     X_f=layer(X_f)
#     print(layer.__class__.__name__,'output shape:\t',X_f.shape)
layerlist_alexnet = []
layer_count = 0
for layer in alexnet_f:
    name = layer.__class__.__name__
    name = name + '_' + str(layer_count)
    layer_count += 1
    layerlist_alexnet.append(name)


print(layerlist_alexnet)
            
    

# layerlist_resnet18 = []
# for layer in resnet18_f:
#     name = layer.__class__.__name__
#     if name == 'Sequential':
#         for l in layer:
#             inner_name = l.__class__.__name__
#             if inner_name == 'Residual18':
#                 layerlist_resnet18.append('residual_'+l.conv1.__class__.__name__+'_1')
#                 layerlist_resnet18.append('residual_'+l.bn1.__class__.__name__+'_1')
#                 layerlist_resnet18.append('residual_'+l.relu1.__class__.__name__+'_1')
#                 layerlist_resnet18.append('residual_'+l.conv2.__class__.__name__+'_2')   
#                 layerlist_resnet18.append('residual_'+l.bn2.__class__.__name__+'_2')
#                 if l.conv3 is not None:  # 确保 conv3 存在
#                     layerlist_resnet18.append('residual_' + l.conv3.__class__.__name__+'_3')      
#                 layerlist_resnet18.append('residual_'+l.relu2.__class__.__name__+'_2')
#             else:
#                 layerlist_resnet18.append(inner_name)
#     else:
#         layerlist_resnet18.append(name)

# print(layerlist_resnet18)

                      



['Conv2d_0', 'ReLU_1', 'MaxPool2d_2', 'Conv2d_3', 'ReLU_4', 'MaxPool2d_5', 'Conv2d_6', 'ReLU_7', 'Conv2d_8', 'ReLU_9', 'Conv2d_10', 'ReLU_11', 'MaxPool2d_12', 'AdaptiveAvgPool2d_13', 'Flatten_14', 'Linear_15', 'ReLU_16', 'Dropout_17', 'Linear_18', 'ReLU_19', 'Dropout_20', 'Linear_21']


### load all the datas:  
    1. FashionMNIST
    2. CIFAR100
    3. CIFAR10

In [31]:
# load the data
# fashion mnist
def get_dataloader_workers():
    """Use 4 processes to read the data.

    Defined in :numref:`sec_utils`"""
    return 4

def load_data_fashion_mnist(batch_size, resize=None):
    """下载Fashion-MNIST数据集, 然后将其加载到内存中

    Defined in :numref:`sec_fashion_mnist`"""
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0, transforms.Resize(resize))
    trans = transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(
        root="../data", train=True, transform=trans, download=True)
    mnist_test = torchvision.datasets.FashionMNIST(
        root="../data", train=False, transform=trans, download=True)
    return (torch.utils.data.DataLoader(mnist_train, batch_size, shuffle=True,
                            num_workers=get_dataloader_workers()),
            torch.utils.data.DataLoader(mnist_test, batch_size, shuffle=False,
                            num_workers=get_dataloader_workers()))

def load_data_cifar100(batch_size, resize=None):
    """Download the Fashion-MNIST dataset and then load it into memory.

    Defined in :numref:`sec_utils`"""
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0, transforms.Resize(resize))
    trans = transforms.Compose(trans)
    # import the cifar100 dataset
    cifar_train = torchvision.datasets.CIFAR100(
        root="../data", train=True, transform=trans, download=True)
    cifar_test = torchvision.datasets.CIFAR100(
        root="../data", train=False, transform=trans, download=True)
    return (torch.utils.data.DataLoader(cifar_train, batch_size, shuffle=True,
                                        num_workers=get_dataloader_workers()),
            torch.utils.data.DataLoader(cifar_test, batch_size, shuffle=False,
                                        num_workers=get_dataloader_workers()))
    
def load_data_cifar10(batch_size, resize=None):
    """Download the Fashion-MNIST dataset and then load it into memory.

    Defined in :numref:`sec_utils`"""
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0, transforms.Resize(resize))
    trans = transforms.Compose(trans)
    # import the cifar100 dataset
    cifar_train = torchvision.datasets.CIFAR10(
        root="../data", train=True, transform=trans, download=True)
    cifar_test = torchvision.datasets.CIFAR10(
        root="../data", train=False, transform=trans, download=True)
    return (torch.utils.data.DataLoader(cifar_train, batch_size, shuffle=True,
                                        num_workers=get_dataloader_workers()),
            torch.utils.data.DataLoader(cifar_test, batch_size, shuffle=False,
                                        num_workers=get_dataloader_workers()))
    

### set the training parameters

In [32]:
batch_size = [128]
epochs = [5]
rounds = 1
lr = 0.01

## Train Function

#### using pynvml to get the GPU power consumption

In [33]:
def nvml_sampling_thread(handle, filename, stop_event, sampling_interval):
    """
    在单独的线程中定期调用 NVML, 获取功耗数据并存储到 data_queue 中。
    参数：
    - handle: nvmlDeviceGetHandleByIndex(0) 得到的 GPU 句柄
    - data_queue: 用于存放 (timestamp, power_in_watts) 数据的队列
    - stop_event: 当此事件被设置时，线程应结束循环
    - sampling_interval: 采样间隔（秒）
    """
    with open(filename/'energy_consumption_file.csv', 'a') as f:  # 追加模式
        # 写入列名
        f.write("timestamp,power_in_watts\n")
        while not stop_event.is_set():
            try:
                # 采集功率和时间戳
                current_time = time.time()
                current_power = pynvml.nvmlDeviceGetPowerUsage(handle) / 1000.0  # 转换 mW -> W
                # 写入文件
                f.write(f"{current_time},{current_power}\n")
                # 等待下一次采样
                time.sleep(sampling_interval)
            except pynvml.NVMLError as e:
                print(f"NVML Error: {e}")
                break

#### set the interval of the power consumption

In [34]:
def integrate_power_over_interval(samples, start_time, end_time):
    # 假定 samples是按时间升序排序的 (t, p)
    # 若未排序，请先排序:
    # samples = sorted(samples, key=lambda x: x[0])
    
    def interpolate(samples, target_time):
        # 在 samples 中找到 target_time 左右最近的两个点，并进行线性插值
        # 若 target_time 恰好等于某个样本点时间，直接返回该点功率
        # 若无法找到两侧点（如 target_time在样本时间轴外），根据情况返回None或边界点
        n = len(samples)
        if n == 0:
            return None
        # 若 target_time 小于第一个样本点时间，无法向左插值，这里直接返回第一个点的功率值(或None)
        if target_time <= samples[0][0]:
            # 简化处理：返回最早样本点的功率（或None）
            return samples[0][1]
        # 若 target_time 大于最后一个样本点时间，无法向右插值，返回最后一个点的功率（或None）
        if target_time >= samples[-1][0]:
            return samples[-1][1]

        # 否则，在中间插值
        # 使用二分查找快速定位
        import bisect
        times = [t for t, _ in samples]
        pos = bisect.bisect_left(times, target_time)
        # pos是使times保持有序插入target_time的位置
        # 因为target_time不在已有样本点中，pos不会越界且pos>0且pos<n
        t1, p1 = samples[pos-1]
        t2, p2 = samples[pos]
        # 线性插值： p = p1 + (p2 - p1)*((target_time - t1)/(t2 - t1))
        ratio = (target_time - t1) / (t2 - t1)
        p = p1 + (p2 - p1)*ratio
        return p

    # 从原始 samples 中筛选出位于[start_time, end_time]内的点
    filtered = [(t, p) for t, p in samples if start_time <= t <= end_time]

    # 如果不足2个点，则尝试使用插值
    if len(filtered) < 2:
        # 无论如何都需要在边界处插值出两个点(起码start和end)
        start_power = interpolate(samples, start_time)
        end_power = interpolate(samples, end_time)

        # 如果从样本中无法插值出任何有意义的点（比如samples为空或无法插值），返回0.0
        if start_power is None or end_power is None:
            return 0.0

        # 将插值的边界点加入到 filtered
        # 注意：如果filtered中有一个点在区间内，我们也需要确保边界有两点以上
        # 例如filtered只有一个点在中间，则需要在start和end插值点全部加入。
        # 若filtered为空，则只用start/end两点插值点求积分
        new_filtered = [(start_time, start_power)] + filtered + [(end_time, end_power)]
        # 确保按时间排序
        new_filtered.sort(key=lambda x: x[0])
        filtered = new_filtered

    # 正常积分计算
    if len(filtered) < 2:
        # 经过插值仍不够，返回0
        return 0.0

    total_energy = 0.0
    for i in range(len(filtered)-1):
        t1, p1 = filtered[i]
        t2, p2 = filtered[i+1]
        dt = t2 - t1
        avg_p = (p1 + p2)/2.0
        total_energy += avg_p * dt

    return total_energy

#### set the training process

In [35]:
def train_func(net, train_iter, test_iter, num_epochs, lr, device, filename, sampling_interval):
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    net.apply(init_weights)
    print('training on', device)
    net.to(device)
    # print(f'The name of the layers are: {alexlayer}')
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()

    # save all epochs time data using list
    to_device_intervals_total = []
    forward_intervals_total = []
    loss_intervals_total = []
    backward_intervals_total = []
    optimize_intervals_total = []
    test_intervals_total = []

    # create a dictionary to store each layer time period data in each batch
    layer_time_alexnet = {}

    # create a list to store the epoch time data
    epoch_intervals_total = []
    
    # 初始化NVML和采样线程
    pynvml.nvmlInit()
    handle = pynvml.nvmlDeviceGetHandleByIndex(0)
    power_data_queue = queue.Queue()
    stop_event = threading.Event()
    sampler_thread = threading.Thread(target=nvml_sampling_thread, args=(handle, filename, stop_event, sampling_interval))
    sampler_thread.start()

    for epoch in range(num_epochs):
        layer_time_alexnet[str(epoch)] = {}
        print('The epoch is:', epoch+1)
        metric = d2l.Accumulator(3)  # train_loss, train_acc, num_examples
        to_device_intervals_epoch = []  # 用来记录本epoch每个batch的to_device时间段
        forward_intervals_epoch = []  # 用来记录本epoch每个batch的forward时间段
        loss_intervals_epoch = []  # 用来记录本epoch每个batch的loss时间段
        backward_intervals_epoch = [] 
        optimize_intervals_epoch = []
        test_intervals_epoch = []   
        epoch_intervals_epoch = []  # 用来记录本epoch的时间段

        epoch_start_time = time.time()

        net.train()
        for i, (X, y) in enumerate(train_iter):
            print('The batch is:', i+1)
            layer_time_alexnet[str(epoch)][str(i)] = {}
            optimizer.zero_grad()
            torch.cuda.synchronize()

            # 记录to_device前后的时间戳
            start_ttd_time = time.time()
            X, y = X.to(device), y.to(device)
            torch.cuda.synchronize()
            end_ttd_time = time.time()
            to_device_intervals_epoch.append((start_ttd_time, end_ttd_time))

            # forward
            start_forward_time = time.time()
            y_hat = X
            layer_count = 0
            for layer in net:
                name = layer.__class__.__name__ # get the name of the layer
                name = name + '_' + str(layer_count)
                layer_count += 1
                if name in layerlist_alexnet:
                    start_layer_time = time.time()
                    y_hat = layer(y_hat)
                    torch.cuda.synchronize()
                    end_layer_time = time.time()
                    layer_time_alexnet[str(epoch)][str(i)][name] = (start_layer_time, end_layer_time)

            # y_hat = net(X)
            # torch.cuda.synchronize()
            end_forward_time = time.time()
            forward_intervals_epoch.append((start_forward_time, end_forward_time))

            # loss
            start_loss_time = time.time()
            l = loss_fn(y_hat, y)
            torch.cuda.synchronize()
            end_loss_time = time.time()
            loss_intervals_epoch.append((start_loss_time, end_loss_time))

            # backward
            start_backward_time = time.time()
            l.backward()
            torch.cuda.synchronize()
            end_backward_time = time.time()
            backward_intervals_epoch.append((start_backward_time, end_backward_time))

            # optimize
            start_optimize_time = time.time()
            optimizer.step()
            torch.cuda.synchronize()
            end_optimize_time = time.time()
            optimize_intervals_epoch.append((start_optimize_time, end_optimize_time))

            with torch.no_grad():
                metric.add(l*X.shape[0], d2l.accuracy(y_hat, y), X.shape[0])
            train_acc = metric[1] / metric[2]

        start_test_time = time.time()
        test_acc = d2l.evaluate_accuracy_gpu(net, test_iter)
        end_test_time = time.time()
        print(f'train acc {train_acc:.3f}, test acc {test_acc:.3f}')
        test_intervals_epoch.append((start_test_time, end_test_time))

        epoch_end_time = time.time()
        epoch_intervals_epoch.append((epoch_start_time, epoch_end_time))

        # data need to be saved
        # add the intervals_epoch to intervals_total
        to_device_intervals_total.append(to_device_intervals_epoch)
        forward_intervals_total.append(forward_intervals_epoch)
        loss_intervals_total.append(loss_intervals_epoch)
        backward_intervals_total.append(backward_intervals_epoch)
        optimize_intervals_total.append(optimize_intervals_epoch)
        test_intervals_total.append(test_intervals_epoch)
        epoch_intervals_total.append(epoch_intervals_epoch)


    # 训练结束后关闭线程
    stop_event.set()
    sampler_thread.join()

    pynvml.nvmlShutdown()

    return to_device_intervals_total, forward_intervals_total, loss_intervals_total, backward_intervals_total, optimize_intervals_total, test_intervals_total, epoch_intervals_total, layer_time_alexnet

## Start to train the model

### set a function to train the model with FashionMNIST datasets

In [36]:
def train_model(main_folder, batch_size, num_epochs, round, lr, device, sample_interval, net):
    print(f'The epoch is set: {num_epochs}, batch is set: {batch_size}, is in {round+1}th running')
    # create the folder to store the data
    # epoch_batch_folder = main_folder/f'E{num_epochs}_B{batch_size}_R{round}'
    sr_number = int(sample_interval*1000)
    epoch_batch_folder = f'E{num_epochs}_B{batch_size}_R{round}_SR{sr_number}_layer'

    data_dir = 'fashion_mnist'
    # data_dir = 'cifar100'
    # data_dir = 'cifar10'

    # the folder path is main_folder/epoch_batch_folder
    folder_path = main_folder/epoch_batch_folder/data_dir
    print(f'The folder path is: {folder_path}')
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    else:
        pass
    
    train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=224)
    # show the shape of the data
    list_of_i = []
    for i, (X, y) in enumerate(train_iter):
        if i < 3:
            print('the shape of the', i, 'batch of the train_iter is:', X.shape)
        else:
            pass
        list_of_i.append(i)
    print(f'The number of batches is: {np.array(list_of_i).shape}')
    to_device_intervals_total, forward_intervals_total, loss_intervals_total,\
          backward_intervals_total, optimize_intervals_total, test_intervals_total, epoch_intervals_total, layer_time_alexnet = train_func(net, train_iter, test_iter, num_epochs, lr, device, folder_path, sample_interval)

    # transfer the data to the numpy array
    to_device_data = np.array(to_device_intervals_total)
    forward_time = np.array(forward_intervals_total)
    loss_time = np.array(loss_intervals_total)
    backward_time = np.array(backward_intervals_total)
    optimize_time = np.array(optimize_intervals_total)
    test_time = np.array(test_intervals_total)
    epoch_time = np.array(epoch_intervals_total)

    # save the layer_time_alexnet, the type is a dictionary, need to be saved as a csv file
    # the first column is the epoch, the second column is the batch, the third column is the layer name, the fourth column is the start time, the fifth column is the end time
    layer_time_alexnet_df = pd.DataFrame.from_dict(layer_time_alexnet)
    


    # print(layer_time_alexnet_df)

    # save the data
    np.save(folder_path/'to_device.npy', to_device_data, allow_pickle=True)
    np.save(folder_path/'forward.npy', forward_time, allow_pickle=True)
    np.save(folder_path/'loss.npy', loss_time, allow_pickle=True)
    np.save(folder_path/'backward.npy', backward_time, allow_pickle=True)
    np.save(folder_path/'optimize.npy', optimize_time, allow_pickle=True)
    np.save(folder_path/'test.npy', test_time, allow_pickle=True)
    np.save(folder_path/'epoch.npy', epoch_time, allow_pickle=True)
    layer_time_alexnet_df.to_csv(folder_path/'layer_time.csv', index=False)


### run the model

In [37]:
# sampling_interval = 0.002 # 2ms
sampling_interval = 0.002 # 1ms
# create the folder to store the data
main_folder = DataList[0]

print('The folder is:', main_folder)
    # find out that if the folder exists in the data path
    # 判断文件是否存在
if main_folder.exists():
    print("文件存在。")
else:
    os.makedirs(main_folder)
    print("文件不存在，已创建。")
    print("文件创建于：", main_folder)
for epoch in epochs:
    for batch in batch_size:
        for round in range(rounds):
            train_model(main_folder, batch, epoch, round, lr, device, sampling_interval, alexnet_f)

The folder is: /root/autodl-tmp/GreenAI/3080/ModelsData/alexnet
文件存在。
The epoch is set: 5, batch is set: 128, is in 1th running
The folder path is: /root/autodl-tmp/GreenAI/3080/ModelsData/alexnet/E5_B128_R0_SR2_layer/fashion_mnist


the shape of the 0 batch of the train_iter is: torch.Size([128, 1, 224, 224])
the shape of the 1 batch of the train_iter is: torch.Size([128, 1, 224, 224])
the shape of the 2 batch of the train_iter is: torch.Size([128, 1, 224, 224])
The number of batches is: (469,)
training on cuda
The epoch is: 1
The batch is: 1
The batch is: 2
The batch is: 3
The batch is: 4
The batch is: 5
The batch is: 6
The batch is: 7
The batch is: 8
The batch is: 9
The batch is: 10
The batch is: 11
The batch is: 12
The batch is: 13
The batch is: 14
The batch is: 15
The batch is: 16
The batch is: 17
The batch is: 18
The batch is: 19
The batch is: 20
The batch is: 21
The batch is: 22
The batch is: 23
The batch is: 24
The batch is: 25
The batch is: 26
The batch is: 27
The batch is: 28
The batch is: 29
The batch is: 30
The batch is: 31
The batch is: 32
The batch is: 33
The batch is: 34
The batch is: 35
The batch is: 36
The batch is: 37
The batch is: 38
The batch is: 39
The batch is: 40
The batch is: 41
The batch is